# Decomposition of the Hopfield Network created by the MRP
GOAL: understand what going on step by step

Here they created two separate Hopfield Network, one for the track reconstruction in the odd modules and one for the track reconstruction in the even modules.

## 1. Modules needed

In [7]:
############################### DEPENDENCIES ##################################
import json
import os
import sys
import contextlib
import io
import numpy as np
import inspect
import os.path
import matplotlib.pyplot as plt
from math import pi, atan, sin, sqrt, tanh, cosh, exp, ceil
import seaborn as sns
from numpy.core.fromnumeric import shape
import random
import time
import math
import statistics
from pathlib import Path

## 2. File path

In [8]:
filename = inspect.getframeinfo(inspect.currentframe()).filename
file_path = os.path.dirname(os.path.abspath(filename))
project_root = os.path.dirname(file_path)

if project_root not in sys.path:
    sys.path.append(project_root)

from event_model import event_model as em
from validator import validator_lite as vl
import data_analysis.event_generator as eg
from visual.color_map import Colormap

print(project_root)

c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub


## 3. Context manager and helper

In [9]:
########################### CONTEXTS ##################################
@contextlib.contextmanager
def nostdout():
    save_stdout = sys.stdout
    sys.stdout = io.BytesIO()
    yield
    sys.stdout = save_stdout

In [10]:
########################### HELPER FUNCTIONS ##################################
def get_polar_coordinates(x, y):
    r = math.sqrt(x ** 2 + y ** 2)
    phi = math.atan2(x, y)
    if phi < 0:
        phi = math.pi - phi
    return r, phi

## 4. Save_experiment fct

In [11]:
def save_experiment(exp_name, exp_num, desc, p, event_file_name, nr_events):
    f = open(project_root + "/algorithms/experiments/" + exp_name + ".txt", "a")
    f.write(
        f"\nExperiment {exp_num}\n\n{desc}\nNumber of events: {nr_events}\nParameters: {p}\n"
    )
    f.close()
    evaluate_events(
        project_root + event_file_name,
        p,
        nr_events,
        True,
        project_root + "/algorithms/experiments/" + exp_name + ".txt",
    )
    f = open(project_root + "/algorithms/experiments/" + exp_name + ".txt", "a")


## 6. Evaluate_events fct

In [12]:
def evaluate_events(file_name, parameters, nr_events=1, plot_event=False, output_file=None):

    json_data_all_events = []
    all_tracks = []
    iter_even = 1
    iter_odd = 1

    all_events = [i for i in range(995)]
    random.seed(40)
    random.shuffle(all_events)
    count = 0
    j = 0
    
    while count < nr_events:
        i = all_events[j]
        j += 1
        print("[INFO] Evaluate Event: %s" % file_name + str(i))
        size = os.path.getsize(file_name + str(i) + ".json")
        json_data_event, modules = load_event(
            file_name + str(i) + ".json", plot_event=False
        )
        max_neurons = 0
        last = 0
        for m in modules[0]:
            n_hits = len(m.hits())
            if last * n_hits > max_neurons:
                max_neurons = last * n_hits
            last = n_hits
        last = 0
        for m in modules[1]:
            n_hits = len(m.hits())
            if last * n_hits > max_neurons:
                max_neurons = last * n_hits
            last = n_hits
        if max_neurons > 2200:
            continue

        print(f"\nstarting instance {count} out of {nr_events}\n")
        start_time = time.time()
        even_hopfield = Hopfield(modules=modules[0], parameters=parameters)
        odd_hopfield = Hopfield(modules=modules[1], parameters=parameters)
        end_time = time.time() - start_time
        print(
            "[INFO] Hopfield Networks initialized in %i mins %.2f seconds"
            % (end_time // 60, end_time % 60)
        )

        try:
            iter_even = even_hopfield.bootstrap_converge(
                bootstraps=parameters["bootstrap_iters"],
                method=parameters["bootstrap_method"],
            )
            iter_odd = odd_hopfield.bootstrap_converge(
                bootstraps=parameters["bootstrap_iters"],
                method=parameters["bootstrap_method"],
            )

            start_time = time.time()
            even_hopfield.mark_bifurcation()
            odd_hopfield.mark_bifurcation()
            even_tracks = even_hopfield.full_tracks()
            odd_tracks = odd_hopfield.full_tracks()
            event_tracks = even_tracks + odd_tracks
            end_time = time.time() - start_time
            print(
                "[INFO] tracks extracted in %i mins %.2f seconds"
                % (end_time // 60, end_time % 60)
            )

            json_data_all_events.append(json_data_event)
            all_tracks.append(event_tracks)

            if plot_event:
                even_hopfield.plot_network_results()
                odd_hopfield.plot_network_results()

            count = count + 1
        except:
            continue

    start_time = time.time()
    if output_file:
        print(output_file)
        sys.stdout = open(output_file, "a")
        print(f"Average number of iterations per convergence: {(iter_even+iter_odd)/2}")
        vl.validate_print(json_data_all_events, all_tracks, return_data=True)
        print("____________________")
        sys.stdout.close()
        sys.stdout = sys.__stdout__
    end_time = time.time() - start_time

    # we could check how many tracks acutally cross the detector sides i guess to identify where some clones come from...
    print(
        "[INFO] validation excecuted in %i mins %.2f seconds"
        % (end_time // 60, end_time % 60)
    )



## 7. Hopfield class

### Functions about Network initialization

In [13]:
parameters = {
        ### NEURONS ###
        "random_neuron_init": True,
        "binary_states": False,  # try it out once maybe but scrap it
        ### WEIGHTS ###
        "ALPHA": 1,
        "BETA": 10,
        "GAMMA": 10,
        "narrowness": 200,
        "constant_factor": 0.9,
        "monotone_constant_factor": 0.9,
        #### UPDATE ###
        "T": 1e-8,  # try to experiment with these rather
        "B": 1e-4,  # try to experiment with these rather
        "T_decay": lambda t: max(1e-8, t * 0.01),  # try to remove these
        "B_decay": lambda t: max(1e-4, t * 0.04),  # try to remove these
        "decay_off": False,  # using this
        "randomized_updates": True,
        "fully_randomized_updates": False,
        #### THRESHOLD ###
        "maxActivation": True,
        "THRESHOLD": 0.2,
        ##### CONVERGENCE ###
        "convergence_threshold": 0.00000005,
        "bootstrap_iters": 10,
        "bootstrap_method": "below_mean",
        ###### BIFURC REMOVAL #####
        "smart": True,
        "only_weight": False,
        "max_activation": False,
        ###### Track prunning #######
        # here we could set the threshold
        "pruning_tr": 0.01,
    }

In [50]:
def init_neurons(tracks: list = None):
        # consider hits inbetween 2 modules as one neuron layer
        # the neurons in N are ordered h(1,1)-h(2,1); h(1,1)-h(2,2); h(1,1)-h(2,3) etc
        
        if p["random_neuron_init"]:
            N = np.random.uniform(size=(modules_count - 1, max_neurons))
        else:
            N = np.ones(shape=(modules_count - 1, max_neurons))

        if tracks:
            N = np.zeros(shape=(modules_count - 1, max_neurons))
            
        for idx, nc in enumerate(neuron_count):  
            #idx is a simple index from 0 to 34 (neuron_count is 35)
            #nc is the value of neuronns count for that index ex: [0,1], [1,3], ..., [34,20] 
            # > in layer 0, we have 1 neurons, in layer 2, we have 3 neurons
            N[idx, nc:] = 0
            # this  sets the value of N matrix to zero for elements after neuron_count[idx] in each row idx of N, 
            # effectively limiting the number of neurons in each layer to the corresponding value in neuron_count[idx].


        N_info = np.zeros(shape=(modules_count - 1, max_neurons, 4))
        
        for idx in range(modules_count - 1):
            m1 = m[idx]       #first module i
            m2 = m[idx + 1]   #second module i+1

            for i, hit1 in enumerate(m1.hits()):
                for j, hit2 in enumerate(m2.hits()):
                    # crossing all hits i in module 1 with all hits j in module 2

                    n_idx = i * hit_counts[idx + 1] + j 
                    
                    if tracks:
                        for t in tracks:
                            if hit1 in t and hit2 in t:
                                N[idx, n_idx] = 1 #activate the neuron
                                
                    # maybe we can check these angles again
                    angle_xz = atan((hit2.x - hit1.x) / (hit2.z - hit1.z))
                    angle_yz = atan((hit2.y - hit1.y) / (hit2.z - hit1.z))
                    norm_dist = sqrt(
                        (hit2.y - hit1.y) ** 2 + (hit2.x - hit1.x) ** 2
                    ) / sqrt(
                        (hit2.z - hit1.z) ** 2
                    )  # does not work!!!

                    _, r_hit1 = get_polar_coordinates(hit1.x, hit1.y)
                    _, r_hit2 = get_polar_coordinates(hit2.x, hit2.y)
                    monotone_dist = (r_hit2 - r_hit1) / (hit2.z - hit1.z)

                    N_info[idx, n_idx, 0] = abs(angle_xz)
                    N_info[idx, n_idx, 1] = abs(angle_yz)
                    N_info[idx, n_idx, 2] = norm_dist  # not robust!!!!
                    N_info[idx, n_idx, 3] = monotone_dist

        return N, N_info

In [51]:
def init_weights(neg_weights=False):

        ### Get parameters from the dictionnary p
        alpha = p["ALPHA"]
        beta = p["BETA"]
        gamma = p["GAMMA"]

        #### Creation of the weights matrix empty
        W = np.zeros(shape=(modules_count - 2, max_neurons, max_neurons)) 

        ### Filling of weights
        for w_idx in range(modules_count - 2):  #This loop iterates w_idx over the indices of the modules, excluding the first and last modules.
            
            for con_idx in range(hit_counts[w_idx + 1]):  # This loop iterates con_idx over the indices of the hits in the current module.
                
                for i in range(hit_counts[w_idx]):  # m1
                    ln_idx = i * hit_counts[w_idx + 1] + con_idx  # left_neuron_idx
                    
                    for j in range(hit_counts[w_idx + 2]):  # m3
                        rn_idx = ( con_idx * hit_counts[w_idx + 2] + j)  # right_neuron_idx

                        # Constant term from the other group
                        constant = (N_info[w_idx, ln_idx, 2] - N_info[w_idx + 1, rn_idx, 2])
                        constant = tanh(constant) * (p["narrowness"] + 1) # tanh to force between -1 and 1
                        constant = (-2 * constant ** 2) + 1  # this should be high if both terms are similar and low/penalizing if both are not similar
                        constant = constant * p["constant_factor"]
                        constant = min(max(constant, - p["constant_factor"]),p["constant_factor"])

                        # monotone constant
                        monotone_constant = (N_info[w_idx, ln_idx, 3]- N_info[w_idx + 1, rn_idx, 3])
                        monotone_constant = tanh(monotone_constant) * (p["narrowness"] + 1)  # tanh to force between -1 and 1
                        monotone_constant = ( -2 * monotone_constant ** 2) + 1  # this should be high if both terms are similar and low/penalizing if both are not similar
                        monotone_constant = (monotone_constant * p["monotone_constant_factor"])
                        monotone_constant = min(max(monotone_constant, - p["monotone_constant_factor"]),p["monotone_constant_factor"],)

                        theta = abs(
                            N_info[w_idx, ln_idx, 0]
                            - N_info[w_idx + 1, rn_idx, 0]
                        )

                        phi = abs(
                            N_info[w_idx, ln_idx, 1]
                            - N_info[w_idx + 1, rn_idx, 1]
                        )

                        W[w_idx, ln_idx, rn_idx] = (
                            alpha
                            * ((1 - sin(theta)) ** beta)
                            * ((1 - sin(phi)) ** gamma)
                            + monotone_constant
                            + constant
                        )
                        #    + constant # this does not work properly

                        if not neg_weights:
                            W[w_idx, ln_idx, rn_idx] = max(
                                0, W[w_idx, ln_idx, rn_idx]
                            )
        return W
                        # maybe we can play a bit around with this



### Functions about network convergence
The "update functions" are used to modify the N matrix containing the state of the neurons. (not the weight matrix !)

In [17]:
def update():
    update_list = []
    for idx in range(modules_count - 1): # for each neuron layer
        c1 = hit_counts[idx]             # number of hits in module 1
        c2 = hit_counts[idx + 1]         # number of hits in module 2
        for i in range(c1 * c2):         # number of neurons created between these two module 
            update_list.append((idx, i)) # create a list of tuples with the idx of the module, and an 'id'
                                         # for the hit, so we'll have by module a number of tuples equals 
                                         # to the number of hits in this module.hits

    if p["randomized_updates"]:
        random.shuffle(update_list)

    if p["fully_randomized_updates"]:
        for c in range(len(update_list)):
            idx, i = random.sample(update_list, 1)[0]
            update_neuron(idx, i)

    else:
        for idx, i in update_list:
            update_neuron(idx, i)

In [18]:
def update_neuron(idx,i): #where idx is the layer number, and i the neuron ID
        b = p["B"]
        t = p["T"]
        c1 = hit_counts[idx]
        c2 = hit_counts[idx + 1]
        update = 0
        
        ### PART 1 ###
        
        if idx > 0:                 # for all layers but not the first
            update += N[idx - 1, :].T @ W[idx - 1, :, i] #update based on the weights of the previous module (idx-1)

        if idx < modules_count - 2: # for all layers but not the two last 
            update += W[idx, i, :] @ N[idx + 1, :] #update based on the weights of this module (idx)
            
        if 0 < idx < modules_count - 1:  # for all layers but not the first or the last
            update /= 2


        ### PART 2 ###
        # left module and right module hit id -> current neuron connects hit lm_id with hit rn_id
        lm_id = i // c2
        rm_id = i % c2


        # all segments mapping to the hit in m1 -> the left module
        m1h = np.sum(N[idx, lm_id * c2 : (lm_id + 1) * c2])
        # all segments mapping to the hit in m2 - the right module
        m2h = np.sum(N[idx, : c1 * c2].reshape(c2, c1)[rm_id, :])  

        # we need to subtract the neuron of the segment 2 times because we add it 2 times
        pen = m1h + m2h - 2 * N[idx, i]

        _update = 0.5 * (1 + tanh(update / t - b * pen / t))

        if p["binary_states"]:
            # NB: usually never use binay states
            if random.random() < _update:
                N[idx, i] = 1
            else:
                N[idx, i] = 0
        else:
            N[idx, i] = _update


In [19]:
def energy():
        b = p["B"]

        E = 0
        bifurc_pen = 0
        for idx in range(modules_count - 2):
            c1 = hit_counts[idx]
            c2 = hit_counts[idx + 1]
            c3 = hit_counts[idx + 2]

            f1 = 0.5
            f2 = 0.5
            if idx == 0:
                f1 = 1
            if idx == modules_count - 3:
                f2 = 1
            N1_pen = N[idx, : c1 * c2].reshape(c2, c1)
            N2_pen = N[idx + 1, : c2 * c3].reshape(c2, c3)
            bifurc_pen = (
                np.sum(np.trace(N1_pen @ N1_pen.T)) * f1
                + np.sum(np.trace(N2_pen @ N2_pen.T)) * f2
                - np.sum(N[idx, :] * N[idx, :]) * f1
                - np.sum(N[idx + 1, :] * N[idx + 1, :]) * f2
            )

            E += (
                -0.5 * (N[idx, :].T @ W[idx, :, :] @ N[idx + 1, :])
                + b * bifurc_pen
            )
        return E

In [20]:
def converge():
        # Basically keep updating until the difference in Energy between timesteps is lower than 0.0005 (Based on Stimfple-Abele)
        # Passaleva uses a different kind of convergence i think (4)
        energies = [energy()]
          # store all energies (not fastest but maybe nice for visualisations)
        t = 0  # timesteps

        p["T"] = start_T
        # self.p["B"] = self.start_B
        # print(f"N at iteration{t}:", np.round(my_instance.N, 1))
        update()
        t += 1
        energies.append(energy())

        while (abs(abs(energies[-2]) - abs(energies[-1]))>= p["convergence_threshold"]):
            update()
            energies.append(energy())
            # print(f"N at iteration{t}:", np.round(my_instance.N, 1))
            t += 1
            if not p["decay_off"]:
                p["T"] = p["T_decay"](p["T"])
                p["B"] = p["B_decay"](t)
            else:
                pass  # keep T and B fixed

        # print("Network Converged after " + str(t) + " steps")
        # print("Energy = " + str(self.energies[-1]))
        return N, energies[-1], t

In [21]:
def bootstrap_converge(bootstraps=50, method="below_mean"):
        start_time = time.time()
        states_list = []
        energy_list = []
        iter_list = []

        for i in range(bootstraps):

            if p["random_neuron_init"]:
                # We only need to reinitialize if we randomly initialize
                init_neurons()

            states, energy, iters = converge()
            print("energy: " + str(energy))

            states_list.append(states)
            energy_list.append(energy)
            iter_list.append(iters)
            # print(f"Finished {i+1}/{bootstraps} iterations")

        if method == "minimum":
            # eventually we could take the lowest 20% or so
            N = states_list[np.argmax(energy_list)]
            energy_list = [np.amax(energy_list)]

        elif method == "below_median":
            median = np.median(energy_list)
            _tmp_states = []
            for states, e in zip(states_list, energy_list):
                if e <= median:
                    _tmp_states.append(states)
            _tmp_states = np.stack(_tmp_states, axis=2)
            N = np.mean(_tmp_states, axis=2)

        elif method == "below_mean":
            mean = np.mean(energy_list)
            _tmp_states = []
            for states, e in zip(states_list, energy_list):
                if e <= mean:
                    _tmp_states.append(states)
            _tmp_states = np.stack(_tmp_states, axis=2)
            N = np.mean(_tmp_states, axis=2)
            
        else:
            stacked_states = np.stack(states_list, axis=2)
            N = np.mean(stacked_states, axis=2)



        end_time = time.time() - start_time
        print(
            "[HOPFIELD] converged network by %s after %i mins %.2f seconds; (energy: %.2f)"
            % (method, end_time // 60, end_time % 60, np.mean(energy_list))
        )
        return sum(iter_list) / len(iter_list)


### Functions about obtaining the final tracks

In [22]:
def tracks():
# What the papers say:  The answer is given by the final set of active Neurons.
# Idea: All sets of Neurons connected together are considered as track candidates.
# Code idea: All neurons that share a hit and are both activated are track candidates.

    global_candidates = []
    global_candidate_states = []

    for idx in range(modules_count - 2): #for each layer of the network but the last one
            candidates = []
            candidate_states = []
            
            l1 = hit_counts[idx]     # number of hits in module 1
            l2 = hit_counts[idx + 1] # number of hits in module 2
            l3 = hit_counts[idx + 2] # number of hits in module 3

            if p["maxActivation"]:
                candidates = []
                thresh = p["THRESHOLD"]

                n1_transform = N[idx, : l2 * l1].reshape(l1, l2).T.copy()       # select the row of the layer idx in the N matrix and the first l2*l1 element (the number of neurons/segments created between module 1 and module 2)
                n2_transform = N[idx + 1, : l3 * l2].reshape(l2, l3).T.copy()   # select the row of the layer idx+1 in the N matrix and the first l3*l2 element (the number of neurons/segments created between module 2 and module 3)
                
                for con in range(l2):  # loop over the "connection hits" in module 2
                    
                    h1_idx = np.argmax(n1_transform[con, :]) #index of the neuron/semgent with maximum activation value in the 'conth' row of n1_transform > select of the left neuron/segment for the conth hit in module 2
                    h3_idx = np.argmax(n2_transform[:, con]) #index of the neuron/segment with maximum activation value in the 'conth' column of n2_transform > select of the right neuron/segment for the conth hit in module 2

                    if (n1_transform[con, h1_idx] < thresh or n2_transform[h3_idx, con] < thresh): 
                        continue #skip this if the value are too low

                    hit1 = m[idx].hits()[h1_idx]     #retrieve the info of the hit in m1 which is the left hit of the segment h1_idx
                    hit2 = m[idx + 1].hits()[con]    #retrieve the info of the hit in m2 'con' (right hit of h1_idx, left hit of h3_idx)
                    hit3 = m[idx + 2].hits()[h3_idx] #retrieve the info of the hit in m3 which is the right hit of the segment h3_idx

                    candidates.append(em.track([hit1, hit2, hit3])) #created a track candidate with the concerned hits
                    extracted_hits.add(hit1) #add the concerned hits
                    extracted_hits.add(hit2)
                    extracted_hits.add(hit3)

                    # storage of the states of neurons concerned by the tracks
                    candidate_states.append(n1_transform[con, h1_idx]) 
                    candidate_states.append(n2_transform[h3_idx, con])


            global_candidates += candidates
            global_candidate_states += candidate_states

    extracted_tracks = global_candidates
    extracted_track_states = global_candidate_states

    return global_candidates


In [23]:
def prune_tracks(tracks, track_infos):
        tr = p["pruning_tr"]
        out_tracks = []
        
        for track, info in zip(tracks, track_infos): #for each track candidates
            num_hits = len(track) #total number of hits by tracks
            if num_hits < 3:      #discard the track candidate that are too small
                continue

            cand = [track[0], track[1]]  #temporary list containing the first two segments in the track candidate
            cand_info = info[0]          #variable contaning additional info associated with the first segment in the track

            for idx in range(1, num_hits - 1): #loops through the remaining segments in the track candidate 
            # and checks if the difference between the additional information for the current segment and 
            # the previous segment is below a pruning threshold (reminder: angles info are stored in there)

                if sum(abs(cand_info - info[idx])) < tr:
                    cand = cand + [track[idx + 1]]  #the current segment is added to the cand list
                    
                else:
                    if len(cand) > 2: #i.e. another segment has been added before in the loop because the tr was small
                        out_tracks = out_tracks + [cand]  #the cand list is added to the out_tracks list, which is a list of pruned track candidates.
                    cand = [track[idx], track[idx + 1]]   #let's go the the next track
                cand_info = info[idx] #let's go to the next track

            if len(cand) > 2: #the cand list is added to the out_tracks list, which is a list of pruned track candidates.
                out_tracks = out_tracks + [cand]
                
        return out_tracks #NB: out_tracks = tracks kept in the final solution


In [24]:
def full_tracks():
        # this will deal with stange angles!!!
        # under the assumption that we removed bifuration completely
        # init this active tracks with all active neurons in layer 1! -> key is the right hit

        global_candidates = []
        global_candidate_states = []
        global_candidate_info = []
        tracks = {}
 
        # Operates in two nested loops over the number of modules and number of hits in each module.
        for idx in range(modules_count - 1): #for each layer

            tracks_2 = {}
            l1 = hit_counts[idx]      # number of hits in module 1 / Left module
            l2 = hit_counts[idx + 1]  # number of hits in module 2 / Right module

            tr = p["THRESHOLD"]

            for segment in range(l1 * l2): #for each neuron/segment in the layer idx

                if N[idx, segment] < tr:    # neurons states matrix
                    continue                # if the activation value is too low, then don't even consider this segment as a candidate
                
                l_hit = m[idx].hits()[segment // l2]
                r_hit = m[idx + 1].hits()[segment % l2] 
                #> it work well to create all combinations of hits inbetween the two modules so all segments possible (same as inside of the function update_neurons())

                ### IF CONTINUING A TRACK ####
                if l_hit in tracks.keys():   #useful once the function has already loop several times, check if the left hit is already in the track dictionary,
                #as we start the track from left to right, the right hit of the track i could become the left hit of the track i+1

                    (track, states, angle) = tracks[l_hit]

                    track = track + [r_hit]                     #save a new track by adding the right hit ot the existing track
                    states = states + [N[idx, segment]]         #add the state of the new segment added to the list of segments states already in the track
                    info = angle + [N_info[idx, segment, :]]    #add the info of the new segment added to the list of segments info already in the track
                    del tracks[l_hit]                           #deleted the previous track as a new one has been created on top of it

                    extracted_hits.add(r_hit) 
                    tracks_2[r_hit] = (track, states, info)
                
                ### IF STARTING A NEW TRACK ####
                else: # i.e. the left hit is not already used in track before
                    
                    track = [l_hit, r_hit]            #save a new track with the left and right hit (only 1 segment in there)
                    states = [N[idx, segment]]        #save the activation value of the segment considered as track 
                    info = [N_info[idx, segment, :]]  #save the angle information of the segment considered as track

                    tracks_2[r_hit] = (track, states, info) #add the right hit to the tracks_2 dictionnary with the new informations > we go from the left to the right of the detector 
                    extracted_hits.add(r_hit)               #add the right hit to the extracted hits set
                    extracted_hits.add(l_hit)               #add the left hit to the extracted hits set

            for _, value in tracks.items():
                (track, states, info) = value
                global_candidates = global_candidates + [track]
                global_candidate_states = global_candidate_states + [states]
                global_candidate_info = global_candidate_info + [info]
            tracks = tracks_2

        for _, value in tracks.items():
            (track, states, info) = value
            global_candidates = global_candidates + [track]
            global_candidate_states = global_candidate_states + [states]
            global_candidate_info = global_candidate_info + [info]

        # here comes the function of 'pruning...' maybe i need to store more info for doing that!!!
        global_candidates = prune_tracks(global_candidates, global_candidate_info)

        global_candidates = [em.track(hits) for hits in global_candidates] # pruned track candidates are then passed through the em.track() function in full_tracks() to produce a list of final extracted tracks
        extracted_tracks = global_candidates
        extracted_track_states = global_candidate_states

        return global_candidates


In [25]:
def mark_bifurcation():
        zero = True
        max_activation = p["max_activation"]
        smart = p["smart"]

        if max_activation:
            zero = False

        if smart:
            zero = False
            max_activation = False

        tr = p["THRESHOLD"]
        N[N <= tr] = 0 # All neurons states values that are less than or equal the treshold are set to 0.

        ## General idea of search for bifurcation neurons:
        # STEP 1: we visit all neurons in one layer and keep going onl with neurons where the activation (state value) is bigger than the treshold
        # STEP 2: we check all neurons for activation and check the ones higher than the treshold
        # STEP 3: for each segment we look wether there is bifurcation on the left or right hit
        # STEP 4: we keep the segment with the highest activation value, put its value to 1 and the activation value of the others segments concerned by the bifurcation to 0


        for idx in range(modules_count - 1): # for each layer of the network

            #STEP 1
            c1 = hit_counts[idx]       #number of hits in module 1
            c2 = hit_counts[idx + 1]   #number of hits in module 2

            for segment in range(c1 * c2): # loop through all possible segments/neurons in layer 1 (number = c1*c2, all hits combinations tested)
                
                # STEP 2
                if N[idx, segment] < tr:   # if neuron state value is under the threshold don't consider this neuron
                    continue
                
                r_hit = segment % c2  # ID of the right hit of the segment      (NB: idx is the same so no need to precise from here)
                l_hit = segment // c2 # ID of the left hit of the segment       (NB: idx is the same so no need to precise from here)

                # STEP 3

                # A. LEFT-RIGHT BIFURCATION (for each pair of left and right hits)

                activation_mask = N[idx, : c1 * c2].reshape(c1, c2)[:, r_hit] > tr   
                # Idea: creation of an activation mask to identify the activated neurons (segments) in the current layer that are connected to a specific neuron (segment) in the next layer, i.e. a neuron using the r_hit

                # Shape: reshaping the row corresponding to the current neuron/segment in the weight matrix N into a 2D array of shape (c1, c2) (with c1 rows and c2 columns in the 2D weight matrix for the current layer) 
                # and then selecting the column corresponding to the next neuron in the weight matrix.
                # Operation: this activation_mask is a 1D Boolean array of length c1 (#hits in the left-module) where each element i is True
                # if the i-th neuron (segment) in the current layer (idx) is activated (i.e. higher than treshold) and is connected to the next neuron (segment) specified by the segment variable

                if sum(activation_mask) > 1:  # we have at least 1 bifurcation into the right hit

                    affected_neurons = []
                    for i in range(c1):  # loop over all nerons affected by the bifurcation
                        if activation_mask[i]:
                            if zero: #condition precised at the start (NB: I think zero is always false ...)
                                N[idx, (i * c2) + r_hit] = 0 #the state of the affected segments set to 0
                            else:
                                affected_neurons = affected_neurons + [(i * c2) + r_hit] # we obtain a list with each element looking like [ID of the segment in this layer + ID of the right hits of the segment]

                    # METHOD 1
                    if smart:
                        # Simple rule: when a bifurcation is detected on right side, we look next active neurons going out
                        # and promote the ones where the weight is high (angle diff is low) if next neuron layer exist

                        if idx < modules_count - 2:  # can check to the right, i.e. there is a next neuron layer on the right

                            c3 = hit_counts[idx + 2] # hits in module 3

                            activation_mask_2 = (N[idx + 1, : c2 * c3].reshape(c2, c3)[r_hit, :] > tr) 
                            # Idea: creation of an activation mask to identify the activated neurons (segments) in the current layer that are connected to a specific neuron (segment) in the next layer, 
                            # i.e. a neuron using the r_hit

                            # Same idea but length c2 and element is True if the i-th neuron (segment) in the next layer (idx+1) is activated (i.e. higher than treshold) and is connected to the next neuron (segment)

                            affected_neurons_2 = []
                            for i in range(c3):  # loop over all nerons affected by bifurcation
                                if activation_mask_2[i]:
                                    affected_neurons_2 = affected_neurons_2 + [c3 * r_hit + i]

                            if len(affected_neurons_2) > 0: 
                                max_val = 0
                                max_l = None
                                max_r = None

                                for e in affected_neurons: # list of ID neuron/segment in layer idx using the r_hit 
                                    for j in affected_neurons_2: # list of ID neuron/segment in layer idx+1 also using the r_hit (but as a left hit lmao)
                                        
                                        c = (N[idx, e] * W[idx, e, j]* N[idx + 1, j]) # new calculated weight 

                                        if p["only_weight"]:
                                            c = W[idx, e, j] #using the new calculated weight 

                                        if c > max_val: #if this new calculated weight is the higher for all j segment of layer idx+1, then ...
                                            max_l = e #Index in affected neuron
                                            max_r = j #Index in afftected neurons 2
                                            max_val = c #new weight
                                            
                                    # a max_l, max_r, max_val are calculated for each loop

                                    N[idx, e] = 0 # state of neuron/segment e of layer idx set up to 0
                                                                        

                                for j in affected_neurons_2:
                                    N[idx + 1, j] = 0 # state of neuron/segment j of layer idx+1 set up to 0

                                #All "concerned" neurons/segments have their weight put back to 0 
                                
                                if max_r is not None and max_l is not None: #i.e. if a pairs of segment has distinguished itself
                                    N[idx, max_l] = 1       #weight of the neuron/segment in idx that obtained the max value c
                                    N[idx + 1, max_r] = 1   #weight of the neuron/segment in idx+1 that obtained the max value c


                            else: #if the right hit is not used as a left hit for some segments in the next layer (i.e no bifurcation in the next module)
                                max_activation = True

                        else: #if it's the last layer
                            max_activation = True
                    

                    # METHOD 2: the method sets the activation state of the neuron with the highest activation value to 1.
                    if max_activation:
                        max_activation = N[idx, affected_neurons[0]] #variable created to obtain the activation values of the neurons in layer idx affected by bifuraction
                        max_id = affected_neurons[0]                 #variable created to keep the ID of neurons with highest activation value
                        
                        for e in affected_neurons: #for segment affected by the bifurcation
                            if N[idx, e] >= max_activation:
                                max_id = e
                                max_activation = N[idx, e]

                            N[idx, e] = 0 # put the activation values of all concerned neurons to 0

                        N[idx, max_id] = 1 # excpect for the one with the highest max activation value that will be set to 1

                    if smart:
                        max_activation = False





                # right-left bifurcation
                activation_mask = N[idx, : c1 * c2].reshape(c1, c2)[l_hit, :] > tr
                if sum(activation_mask) > 1:
                    affected_neurons = []
                    affected_neurons_2 = []
                    for i in range(c2):
                        if activation_mask[i]:
                            if zero:
                                N[idx, (l_hit * c2) + i] = 0
                            else:
                                affected_neurons = affected_neurons + [(l_hit * c2) + i]
                    if smart:  # i know there can be only one neuron on the right.
                        if idx > 0:
                            # can check here only for the first active neuron, because we removed
                            # the other bifurcation in the previous iteration
                            c0 = hit_counts[idx - 1]
                            activation_mask_2 = (
                                N[idx - 1, : c0 * c1].reshape(c0, c1)[:, l_hit]
                                > tr
                            )
                            if sum(activation_mask_2) > 0:
                                for i in range(
                                    c0  # this was c0...
                                ):  # loop over all nerons affected by bifurc
                                    if activation_mask_2[i]:
                                        affected_neurons_2 = affected_neurons_2 + [
                                            c1 * i + l_hit
                                        ]

                                if len(affected_neurons_2) > 0:
                                    max_val = 0
                                    max_l = None
                                    max_r = None
                                    for e in affected_neurons_2:
                                        for j in affected_neurons:
                                            c = (
                                                N[idx - 1, e]
                                                * W[idx - 1, e, j]
                                                * N[idx, j]
                                            )
                                            if p["only_weight"]:
                                                c = W[idx - 1, e, j]
                                            if c > max_val:
                                                max_l = e
                                                max_r = j
                                                max_val = c
                                        N[idx - 1, e] = 0
                                for e in affected_neurons:
                                    N[idx, j] = 0
                                if max_r is not None and max_l is not None:
                                    N[idx - 1, max_l] = 1
                                    N[idx, max_r] = 1
                            else:
                                max_activation = True
                        else:
                            max_activation = True
                        pass

                    if max_activation:
                        # check to the left or max score
                        max_activation = N[idx, affected_neurons[0]]
                        max_id = affected_neurons[0]
                        for e in affected_neurons:
                            if N[idx, e] >= max_activation:
                                max_id = e
                                max_activation = N[idx, e]
                            N[idx, e] = 0
                        N[idx, max_id] = 1

        # converged, averaged neuron state
        # what do we want to do -> search all neurons wether there is bifurcation
        # how to search this, by the indices... and then we store it as a combination of hit id_s, and which side of this element the bifurcation occurs
        # bifiurcation is stored as a list of hit ids, with


### Functions about plotting the final tracks

In [28]:
def show_all_tracks(threshold=None, show_states=False):
        # Creates a colormap from blue to red for small to large values respectively
        c_map = Colormap(0, 1, 2 / 3.0, 0)
        c = []
        tracks = []
        for idx in range(modules_count - 1):
            m1 = m[idx]
            m2 = m[idx + 1]

            for i, hit1 in enumerate(m1.hits()):
                for j, hit2 in enumerate(m2.hits()):
                    n_idx = i * hit_counts[idx + 1] + j
                    if threshold:
                        if N[idx, n_idx] >= threshold:
                            tracks.append(em.track([hit1, hit2]))
                            if show_states:
                                c.append(c_map.get_color_rgb(N[idx, n_idx]))
                        continue
                    if show_states:
                        c.append(c_map.get_color_rgb(N[idx, n_idx]))
                    tracks.append(em.track([hit1, hit2]))
        eg.plot_tracks_and_modules(tracks, m, colors=c)

def tracks_with_hit(hit):
        return [track for track in extracted_tracks if hit in track.hits]

def print_neurons():
        n = len(N)
        for i in range(n):
            m = int(sqrt(len(N[i])))
            for j in range(m):
                print(f"m{i+1}h{j+1}: {N[i, (j*m):((j+1)*m)]}")


In [27]:
def plot_network_results(show_states=False):
        if show_states:
            # Creates a colormap from blue to red for small to large values respectively
            c_map = Colormap(0, 1, 2 / 3.0, 0)
            colors = []
            [colors.append(c_map.get_color_rgb(v)) for v in extracted_track_states]
            eg.plot_tracks_and_modules(
                extracted_tracks,
                m,
                colors=colors,
                title="Hopfield Output with states",
            )
        else:
            eg.plot_tracks_and_modules(
                extracted_tracks, m, title="Hopfield Output"
            )

In [35]:
def load_event(file_name, plot_event=False):
    f = open(file_name)
    json_data_event = json.loads(f.read())

    ev = em.event(json_data_event, read_tracks=True)

    modules = ev.modules
    tracks = ev.real_tracks

    if plot_event:
        eg.plot_tracks_and_modules(tracks, modules, title="Loaded Event")

    modules_even = []
    modules_odd = []

    for i in range(len(modules)):
        if i % 2 == 0:
            modules_even.append(modules[i])
        else:
            modules_odd.append(modules[i])

    return json_data_event, (modules_even, modules_odd)

In [53]:
def Hopfield_init(modules: list, parameters: dict, tracks: list = None):
    p = parameters
    m = modules
    start_T = p["T"]
    start_B = p["B"]
    N = None
    N_info = None
    modules_count = len(modules)
    hit_counts = [len(module.hits()) for module in m]
    neuron_count = [
        hit_counts[i] * hit_counts[i + 1]
        for i in range(modules_count - 1)
        ]
    flips = 0
    max_neurons = max(neuron_count)
    init_neurons(tracks=tracks)
    init_weights()
    extracted_hits = set()
    extracted_tracks = []
    extracted_track_states = []
    energies = []


### Run fct

In [46]:
# EVALUTE EVENT FOR 1 EVENT WHERE WE CAN TRACK WHAT'S GOING ON

i = 3
event_file_name = "/datasets/minibias/velo_event_"
file_name = project_root + event_file_name
nr_events=1
plot_event=False

json_data_all_events = []
all_tracks = []
iter_even = 1
iter_odd = 1

nr_max_neurons_tracking = []
id_events_tracking = []
total_hits_tracking = []
timing_tracking = []
start_time_networks = time.time() 

print("[INFO] Evaluate Event: %s" % file_name + str(i))

[INFO] Evaluate Event: c:\Users\aurel\Documents\GitHub\Code_Thesis_GitHub\Code_Thesis_GitHub/datasets/minibias/velo_event_3


In [47]:
### NETWORK INITIALIZATION (Activation states and weights matrices)

# STEP 1

start_timing = time.time() 
json_data_event, modules = load_event(file_name + str(i) + ".json", plot_event=False)
            
total_hits = 0
max_neurons = 0
last = 0

for m in modules[0]:
    n_hits = len(m.hits())
    if last * n_hits > max_neurons:
            max_neurons = last * n_hits
    last = n_hits
    total_hits = total_hits + n_hits
            
last = 0
for m in modules[1]:
    n_hits = len(m.hits())
    if last * n_hits > max_neurons:
        max_neurons = last * n_hits
    last = n_hits
    total_hits = total_hits + n_hits


nr_max_neurons_tracking.append(max_neurons)
total_hits_tracking.append(total_hits)
id_events_tracking.append(i)


print(f"\nStarting instance {1} out of {nr_events}\n")
print(f'Number of total hits: {total_hits}')
print(f'Number of max neurons: {max_neurons}\n')


Starting instance 1 out of 1

Number of total hits: 3172
Number of max neurons: 7832



In [52]:
# STEP 2

start_time = time.time()
even_hopfield = Hopfield_init(modules=modules[0], parameters=parameters)
odd_hopfield = Hopfield_init(modules=modules[1], parameters=parameters)
end_time = time.time() - start_time
            
print("[INFO] Hopfield Networks initialized in %i mins %.2f seconds"
        % (end_time // 60, end_time % 60))

[INFO] Hopfield Networks initialized in 0 mins 0.00 seconds
